In [240]:
from BRScraper import nba
import pandas as pd
import re
import time

In [241]:
df = pd.read_csv('contracts.csv')

In [242]:
def get_player_stats(name):
    
    name2 = re.sub('[^a-zA-Z0-9 \n\.]', '', name)

    if name2!=name:
        raise ValueError(name+' has special characters and is not a valid name. Try replacing the special characters.')

    try:
        name_url = name.lower().strip().split(' ')

        if len(name_url[1])>=5:
            len_name = 5
        else:
            len_name = len(name_url[1])

        name_url = name_url[1][:len_name] + name_url[0][:2]

        first_l = name.strip().split(' ')[1][0].lower()

        url = 'https://www.basketball-reference.com/players/'+first_l+'/'+name_url+'01.html'

    except:
        raise ValueError(name+''' is not in a valid name format.
                        Valid names would be "LeBron James", "lebron james" or "LEBRON JAMES" for example.''')
                         
    try:
        player_df = pd.read_html(url)[0]
        
    except:
        raise ValueError(name+' is not a valid name. Check for mispelling errors or if that players exists.')
                         
    player_df = player_df.dropna(how='all', axis=0)
    
    num = player_df[player_df['Season']=='Career'].index[0]
    
    player_df = player_df.iloc[:num]
    
    return player_df

In [243]:
seasons = ['2023-24', '2022-23', '2021-22', '2020-21', '2019-20', '2018-19']

def get_final_data(name: str, contract_year: str):
    player = get_player_stats(name)

    cy_index = seasons.index(contract_year)
    cy = seasons[cy_index]
    cyM1 = seasons[cy_index + 1]
    cyM2 = seasons[cy_index + 2]

    player = player.iloc[::-1]
    player = player[player.Lg.isin(['NBA'])]
    player = player[player.Season.isin([cy, cyM1, cyM2])]
    player = player.drop_duplicates(subset = ['Season'], keep = 'last')
    player = player.drop(['Age', 'Tm', 'Lg', 'Pos', 'Awards',], axis = 1)

    player_contract = df.loc[df['PLAYER'] == 'Steven Adams']
    cy_data = player[player['Season'] == cy]
    cyM1_data = player[player['Season'] == cyM1]
    cyM2_data = player[player['Season'] == cyM2]

    for col in cy_data.columns:
        cy_data.rename(columns={col: 'CY ' + col}, inplace = True)

    for col in cyM1_data.columns:
        cyM1_data.rename(columns={col: 'CYM1 ' + col}, inplace = True)

    for col in cyM2_data.columns:
        cyM2_data.rename(columns={col: 'CYM2 ' + col}, inplace = True)

    cy_data = cy_data.reset_index(drop = True)
    cyM1_data = cyM1_data.reset_index(drop = True)
    cyM2_data = cyM2_data.reset_index(drop = True)
    player_contract = player_contract.reset_index(drop = True)

    final_df = player_contract.join([cy_data, cyM1_data, cyM2_data])

    return final_df

In [249]:
finished_df = pd.DataFrame()

try:
    for index, row in df.iterrows():
        player_info = get_final_data(row['PLAYER'], row['SEASON'])
        finished_df = finished_df.join(player_info)

        time.sleep(5)

except Exception as e:
    print(e)

finished_df.to_csv('dataset.csv', sep = ',' , decimal = '.', index = False)

C:\Users\nsalo\AppData\Local\Temp\ipykernel_19232\1268446808.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cy_data.rename(columns={col: 'CY ' + col}, inplace = True)
C:\Users\nsalo\AppData\Local\Temp\ipykernel_19232\1268446808.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cyM1_data.rename(columns={col: 'CYM1 ' + col}, inplace = True)


Malik Shake Milton is not a valid name. Check for mispelling errors or if that players exists.
